In [1]:
import dwavebinarycsp
import dimod
import neal
import numpy as np

In [8]:
import neal
solver = neal.SimulatedAnnealingSampler()
sampleset = solver.sample_ising(
    {'a': -10.5, 'b': 1.22}, {('a', 'b'): -1.735}, num_reads=100)
sampleset.first.sample["a"] == sampleset.first.sample["b"] == -1

False

In [9]:
def create_linear(vector):
    N = len(vector)
    linear_coeff = {}
    for k in range(N):
        key = 'x' + str(k+1)
        value = vector[k]
        linear_coeff[key] = value
    return linear_coeff

In [71]:
def create_quadratic(array):
    N, _ = np.shape(array)
    quadratic_coeff = {}
    for i in range(N):
        for j in range(i):
            key = ('x' + str(i+1) , 'x' + str(j+1))
            vaule = array[i][j]
            quadratic_coeff[key] = vaule
    return quadratic_coeff

In [84]:
Linear = create_linear(np.random.randn(5))
Quad = create_quadratic(np.random.randn(5, 5))
print(Linear)
print(Quad)

{'x1': 1.313908489779249, 'x2': 0.021568587014796133, 'x3': 0.6964958106440572, 'x4': -0.7848938123697269, 'x5': -1.3845482733953005}
{('x2', 'x1'): 1.3506573220100624, ('x3', 'x1'): -1.0592711700863569, ('x3', 'x2'): -0.671945275881151, ('x4', 'x1'): 0.13050603772055805, ('x4', 'x2'): -0.42621558379935826, ('x4', 'x3'): 0.6245396102608163, ('x5', 'x1'): -0.5435148875729505, ('x5', 'x2'): 2.116224615066441, ('x5', 'x3'): 1.8431301767464507, ('x5', 'x4'): -0.8351859624642974}


In [85]:
import dimod
bqm = dimod.BinaryQuadraticModel(Linear, Quad, 0, 'BINARY')

In [86]:
from dimod.reference.samplers import ExactSolver
sampler = ExactSolver()
response = sampler.sample(bqm)
print(response)

   x1 x2 x3 x4 x5   energy num_oc.
16  0  0  0  1  1 -3.004628       1
17  1  0  0  1  1 -2.103728       1
31  0  0  0  0  1 -1.384548       1
19  0  1  0  1  1 -1.29305       1
12  0  1  0  1  0 -1.189541       1
15  0  0  0  1  0 -0.784894       1
30  1  0  0  0  1 -0.614155       1
11  0  1  1  1  0 -0.540451       1
0   0  0  0  0  0      0.0       1
22  1  0  1  1  1 0.001166       1
3   0  1  0  0  0 0.021569       1
4   0  1  1  0  0 0.046119       1
23  0  0  1  1  1 0.159538       1
8   0  0  1  1  0 0.536142       1
14  1  0  0  1  0 0.659521       1
7   0  0  1  0  0 0.696496       1
28  0  1  0  0  1 0.753245       1
25  1  0  1  0  1   0.8662       1
9   1  0  1  1  0 0.921285       1
6   1  0  1  0  0 0.951133       1
18  1  1  0  1  1 0.958507       1
24  0  0  1  0  1 1.155078       1
10  1  1  1  1  0  1.19535       1
20  0  1  1  1  1  1.19917       1
1   1  0  0  0  0 1.313908       1
13  1  1  0  1  0 1.605531       1
5   1  1  1  0  0 1.651414       1
21  1  1  1  

In [82]:
from dwave.cloud import Client
client = Client.from_config(
    token='')
client.get_solvers()

[Solver(id='DW_2000Q_6')]

In [90]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=1000)
print(sampleset)   # doctest: +SKIP

  x1 x2 x3 x4 x5    energy num_oc. chain_.
0  0  0  0  1  1 -3.004628     994     0.0
1  1  0  0  1  1 -2.103728       4     0.0
2  0  0  0  0  1 -1.384548       1     0.0
3  1  0  1  1  1  0.001166       1     0.0
['BINARY', 4 rows, 1000 samples, 5 variables]
